# Python 中級編２：再帰処理１

## 目次
* [再帰関数](#再帰関数)
 * [練習11.1: フィボナッチ数列](#練習11.1:-フィボナッチ数列)
 * [練習11.2: 最大公約数](#練習11.2:-最大公約数)
 * [練習11.3: 二項係数](#練習11.3:-二項係数)
 * [復習：スライス記法](#復習：スライス記法)
 * [練習11.4: 順列の生成](#練習11.4:-順列の生成)
* [メモ化](#メモ化)
 * [練習11.5: 2項係数の計算のメモ化](#練習11.5:-2項係数の計算のメモ化)
* [練習11.6: 余因子展開による行列式の計算](#練習11.6:-余因子展開による行列式の計算)
* [エクストラ課題11.7: 順列の符号を用いた行列式の計算](#エクストラ課題11.7:-順列の符号を用いた行列式の計算)
 * [Step 1: 順列の符号の計算](#Step-1:-順列の符号の計算)
 * [Step 2: 定義どおりの行列式の計算](#Step-2:-定義どおりの行列式の計算)
* [エクストラ課題11.8: 行列式計算の効率化](#エクストラ課題11.8:-行列式計算の効率化)

---

このノートブックでは，再帰すなわち「自分自身を呼び出す関数」を用いたプログラミングについて学習する．

## 再帰関数

まず階乗 $n! = n \cdot (n-1) \cdots 3 \cdot 2 \cdot 1$ を例として再帰関数とはどういうものか説明する．

$n!$ を $n$ の関数と考え，$f(n) = n!$ と書くことにする．このとき
$$
\begin{align}
f(n) &= n! \\
     &= n \cdot (n-1) \cdot \cdots \cdot 2 \cdot 1 \\
     %&= n \cdot \left\{ (n-1) \cdot \cdots \cdot 2 \cdot 1 \right\} \\
     &= n \cdot (n-1)! \\
     &= n \cdot f(n-1)
\end{align}
$$
と書ける．ここで $n = 1$ の場合も上式が成立するように，$0! = 1$ すなわち $f(0) = 1$ と考えることにしよう．

以上をまとめた
$$
f(n) = \left\{ \begin{array}{cl}
n\cdot f(n-1) & (n \ge 1) \\
1 & (n = 0)
\end{array}
\right.
$$
は，左辺の関数 $f(n)$ の値を $n = 0, 1, 2, \dots$ に対して定義するために，右辺で $f$ それ自体を用いている，と見ることができる．

このように，関数の定義に「いま定義しようとしている関数それ自体」を使うものを「再帰的定義（あるいは帰納的定義）」という．

これが定義として意味を持つためのポイントは，左辺を右辺で書き換えることによる計算
$$
\begin{align}
f(n) &= n\cdot f(n-1) \\
     &= n\cdot(n-1)\cdot f(n-2) \\
     &= n\cdot(n-1)\cdot (n-2) \cdot f(n-3) \\
     &= \cdots
\end{align}
$$
は，いつか $f(0) = 1$ という特別ケースに到達し，そこで必ず止まることである．

再帰的に定義された関数は，そのままプログラムで表現できることが多い．例えば階乗を計算する関数は以下のように定義できる：

In [ ]:
def f(n):
    if n == 0:
        return 1
    else:
        return n * f(n - 1)

上の場合分けによる定義とほぼ同じ形になっていることが分かるだろう．

上の `f` を使って，$100!$ を計算してみなさい：

In [ ]:
f(100)

階乗はもちろんループを使っても実装できる：

In [ ]:
def f_by_loop(n):
    ret = 1
    for k in range(1, n+1): # k = 1, 2, ..., n まで
        ret *= k
    return ret

print(f_by_loop(100)) # 100! をループで計算
print(f(100) == f_by_loop(100)) # 再帰定義の実装と比較

しかし，少し複雑な再帰的定義になると，ループを使って実装するよりも，再帰的定義をそのままプログラムに移し替えるほうがずっと簡単になることが多い．

### 練習11.1: フィボナッチ数列
フィボナッチ数列 $a_1 = 1, a_2 = 1, a_3 = 2, a_4 = 3, a_5 = 5, \dots$ の第 $n$ 項 $a_n$ を計算する関数 `fib(n)` を再帰を使って実装しなさい．

ヒント（ダブルクリックで表示）
<!--
フィボナッチ数列 a_n は場合分けにより

      {  1  (n = 1 のとき)
a_n = {  1  (n = 2 のとき)
      { a_{n-1} + a_{n-2} （それ以外）

と定義できる．
-->

In [ ]:
def fib(n):
    # *** 実装しなさい ***


実装できたらテストしなさい：

In [ ]:
print(fib(1) == 1)
print(fib(2) == 1)
print(fib(3) == 2)
print(fib(20) == 6765)

### 練習11.2: 最大公約数
[ユークリッドの互除法](https://ja.wikipedia.org/wiki/ユークリッドの互除法)を用いて自然数 $n$ と $m$ の最大公約数を求める関数 `gcd(n, m)` を実装しなさい．

ヒント（ダブルクリックで表示）
<!--
ユークリッドの互除法の原理は，
n >= m に対して, n を m で割った余りを r とするとき

n と m の最小公倍数 = {  m (r = 0 のとき）
                   {  m と r の最小公倍数 (r > 0 のとき)
                      
となることだった．
よって r = 0 のときは m を返し, r > 0 のときは再帰的に gcd を呼び出すように実装すればよい．

「n を m で割った余り」は Python では n % m で計算できる．
-->

In [ ]:
def gcd(n, m):
    # *** 実装しなさい ***


実装できたらテストしなさい：

In [ ]:
print(gcd(20, 12) == 4)
print(gcd(12, 20) == 4)
print(gcd(10, 10) == 10)
print(gcd(1, 10) == 1)
print(gcd(1, 1) == 1)

### 練習11.3: 二項係数
2項係数 ${}_n C_k$（$n \ge k \ge 0$）は，階乗を用いて
$$
{}_n C_k = \frac{n!}{k! (n-k)!}
$$ 
と定義される（$0! = 1$ とする）が，以下のように再帰的に定義することもできる：
$$
{}_n C_k = \left\{ \begin{array}{cl}
{}_{n-1} C_k + {}_{n-1} C_{k-1} & (0 < k < n) \\
1 & (それ以外)
\end{array} \right.
$$
**この再帰的定義に従って** ${}_n C_k$ を計算する関数 `comb(n, k)` を実装せよ：

In [ ]:
def comb(n, k):
    # *** 実装しなさい ***


実装できたらテストしなさい：

In [ ]:
# テスト用に，ここでも階乗を定義する
def factorial(n): return n * factorial(n-1) if n > 0 else 1

print(comb(5, 3) == factorial(5) // (factorial(3) * factorial(5 - 3)))
print(comb(20, 10) == factorial(20) // (factorial(10) * factorial(20 - 10)))
print(comb(10, 10) == 1)
print(comb(10, 0) == 1)
print(comb(0, 0) == 1)

### 復習：スライス記法
だいぶ前のノートブック（入門編４：文字列）でスライス記法というのが出てきた．

スライス記法は，文字列やリストやタプルに対して，添え字演算子 `[]` の中に `:` で区切って `s[i:j]` のように「はじめ `i`」と「おわりの1つ先 `j`」の添え字番号を書くことで，文字列やリストの一部を切り出す働きをする．

例えば `xs = [0, 10, 20, 30]` というリストがあるとき
* `xs[i:j]` は `xs` の `i` 番目から `j-1` 番目までの部分リストを取り出す（先頭の要素が 0 番目）．例えば `xs[1:3] = [10, 20]` である．
* 「おわり」を書かずに `xs[i:]` とすると，`i` 番目から最後までの部分リストを取り出す．例えば `xs[2:] = [20, 30]` である．
* 「はじめ」を書かずに `xs[:j]` とすると，先頭から `j-1` 番目までの部分リストを取り出す．例えば `xs[:3] = [0, 10, 20]` である．

スライス記法を使うと，入力されたリスト `xs = [x1, x2, ..., xn]` の順序を逆にした `[xn, ..., x2, x1]` を返す関数 `rev(xs)` は再帰を用いて以下のように実装できる．

In [ ]:
def rev(xs):
    if xs == []:
        return []
    else:
        return rev(xs[1:]) + [xs[0]]

# テスト
rev([1, 2, 3])

上の定義では以下のように `rev` の呼び出しが繰り返されることでリストの逆順が計算される：
```python
 rev([a, b, c, d, ..., x]) 
=   rev([b, c, d, ..., x]) + [a]
=      rev([c, d, ..., x]) + [b] + [a]
=         rev([d, ..., x]) + [c] + [b] + [a]
=            ...
=                  rev([]) + [x] + ... + [c] + [b] + [a]
```

上記の関数 `rev` は特に効率がよいわけではないが，スライス記法と組み合わせた再帰の考え方は例えば以下のような課題で役に立つ．

### 練習11.4: 順列の生成
$1, 2, \dots, n$ を並べ替えた順列は全部で $n!$ 種類ある．各順列をリストで表し，全ての順列のリストを（すなわちリストのリストとして）返す関数 `perm(n)` を実装しなさい．

実行例：
```python
perm(2)
--> [[1, 2], [2, 1]]

perm(3)
--> [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]
```

順列のリスト内で，各順列の現れる順序は上の例と同じでなくてもよい．

$n \ge 2$ のとき `perm(n)` は `perm(n-1)` を使って以下のように定義できる：
```python
perm(n) = perm(n-1)の各順列 [a1, a2, a3, ...] に対し，
          0番目，1番目，..., n-1番目の要素として n を挿入したものを全て集めたリスト
```

例えば `perm(2)` の順列 `[1, 2]` からは，各位置に `3` を挿入することで
```python
[3, 1, 2] =     [] + [3] + [1, 2]
[1, 3, 2] =    [1] + [3] + [2]
[1, 2, 3] = [1, 2] + [3] + []
```
の３つの順列が得られ，同様に `[2, 1]` の各位置に `3` を挿入することで
```python
[3, 2, 1] =     [] + [3] + [2, 1]
[2, 3, 1] =    [2] + [3] + [1]
[2, 1, 3] = [2, 1] + [3] + []
```
の３つの順列が得られる．`perm(3)` はこれら6つの順列を集めたリストになる．

`perm(n-1)` の順列の一つ $p = [a_1, a_2, \dots, a_{n-1}]$ の `i` 番目の要素として `n` を挿入するには，上の例の `=` の右辺に書いたように，

($p$ の $i$ 番目より前の部分)  +  $[n]$   +   ($p$ の $i$ 番目以降の部分)

という風に連結すればよい．これはスライス記法を使って
```python
p[:i] + [n] + p[i:]
```
と書ける．

上の考え方に基づき，`perm(n)` を再帰的に定義しなさい．

<!--
ヒント：`perm(n)` を `perm(n-1)` を使って定義するのはやや難しい．それよりも，与えられたリスト `xs = [a, b, ..., x]` に対する全ての並べ替えを返す関数 `perm_sub(xs)` を再帰的に実装し，それを用いて `perm(n)` を定義するほうが簡単だろう．リスト `[1, 2, ..., n]` はリスト内包記法を使えば `[k for k in range(1, n+1)]` で作れる．
-->

さらにヒント（ダブルクリックで表示）
<!--

* perm(1) は特別ケースとして [[1]] を返せばよい．
  *「長さ 1 の順列 [1] だけを含むリスト」だから [1] ではなく [[1]] であることに注意．

* n >= 2 のときは再帰的に perm(n-1) を呼び出して，その返り値を ps とすれば

  ret = [] # perm(n) の結果をためるリスト

  for p in ps: # perm(n-1) のそれぞれの順列について
      for i in range(??, ??): # ?? は自分で考える
          r = p に i 番目の要素として n を挿入したもの
          ret.append(r)

  return ret

  と計算すればよい
-->

In [ ]:
def perm(n):
    # *** 実装しなさい ***


実装できたらテストしなさい：

In [ ]:
print(perm(2) == [[1,2], [2,1]] or perm(2) == [[2,1], [1,2]])

# xs の順序を無視したオブジェクト（set）を作る
# この部分を理解する必要はない．
def setify(xs): return set(map(tuple, xs))

p4 = [[1, 2, 3, 4], [1, 2, 4, 3], [1, 3, 2, 4],
      [1, 3, 4, 2], [1, 4, 2, 3], [1, 4, 3, 2],
      [2, 1, 3, 4], [2, 1, 4, 3], [2, 3, 1, 4],
      [2, 3, 4, 1], [2, 4, 1, 3], [2, 4, 3, 1],
      [3, 1, 2, 4], [3, 1, 4, 2], [3, 2, 1, 4],
      [3, 2, 4, 1], [3, 4, 1, 2], [3, 4, 2, 1],
      [4, 1, 2, 3], [4, 1, 3, 2], [4, 2, 1, 3],
      [4, 2, 3, 1], [4, 3, 1, 2], [4, 3, 2, 1]]

print(setify(perm(4)) == setify(p4) and len(perm(4)) == 24)

## メモ化
再帰で実装した関数は，計算効率がよくない場合がある．

例えば上で実装したフィボナッチ数列の第`n`項を計算する `fib(n)` で，第36項を計算してみなさい：

In [ ]:
%%time
fib(36)

おそらく数秒はかかっただろう．

`%%time` はマジックコマンドと呼ばれる Jupyter の機能のひとつで，セルの実行時間を表示する．

出力の1行目の
```
CPU times: user 3.33 s, sys: 17.3 ms, total: 3.35 s
```
という表示（数値は人によって，かつ実行のたびに異なる）は，セルの実行のためにコンピュータ（の CPU）が計算した時間が 3.35 秒で，そのうち17.3ミリ秒は Python のプログラムではなく OS の処理に使われたことを意味する．

2行目の
```
Wall time: 3.37 s
```
という表示はセルの実行の開始から終了まで，CPUがこのセルの計算以外の他の仕事をしていた時間も含めて全部で 3.37 秒かかったことを意味する．

さて，上のセルの計算時間のうち大部分は，例えば
```python
fib(10)
--> fib(9) + fib(8)
--> (fib(8) + fib(7)) + (fib(7) + fib(6))
--> (fib(7) + fib(6)) + (fib(6) + fib(5)) + (fib(6) + fib(5)) + (fib(5) + fib(4))
--> ...
```
のように， `fib(n)` の計算の過程で何度も繰り返し同じ項を計算することで消費されている．

例えば上の `fib(10)` の計算過程には `fib(8)` が 2回，`fib(7)` が3回現れているが，それぞれ，毎回同じ値を返すに決まっているのだから，何度も同じ計算をするのは無駄である．

これを防ぐテクニックとして**メモ化**(memoization)と呼ばれるものがある．

これは，まず最初に空の辞書 `memo = {}` を用意しておき，再帰関数の呼び出しのときに，引数 `n` と一緒に辞書 `memo` を渡して
* もしキー `n` が `memo` に存在すれば，値を計算するのではなく `memo`　に記録してある値を単に return する
* もしキー `n` が `memo` に存在しなければ，再帰を使って関数値を計算し，それを `memo` に記録した上で関数値を return する

というやり方である．これによって，同じ引数に対する関数値の計算は一度だけ起きることになる．

以下は，メモ化を使った `fib(n)` の改良版 `fast_fib(n)` である．

In [ ]:
def fast_fib(n):
    # 空の辞書を用意
    memo = {}
    # 本体を呼び出す
    return fib_memo(n, memo)

# メモを使った計算の本体
def fib_memo(n, memo):
    if n == 1 or n == 2: # 特別ケース：初項と第２項
        return 1
    
    if n in memo:
        # メモに計算ずみの n 項目の値があればそれを返す
        return memo[n]
    else:
        # n 項目を再帰で計算
        a = fib_memo(n-1, memo) + fib_memo(n-2, memo)
        memo[n] = a # メモに記録してから
        return a    # 計算結果を返す

さっきと同じ36項目を時間を測りつつ計算してみよう：

In [ ]:
%%time
fast_fib(36)

おそらく一瞬で結果が返ってきただろう．もっと先の項まで計算できるかどうか，やってみなさい．

### 練習11.5: 2項係数の計算のメモ化
再帰を使った2項係数の計算にも `fib(n)` と同様の問題がある．

例えば ${}_{30}C_{15}$ を `comb` で計算してみなさい：

In [ ]:
comb(30, 15)

当分おわらないので，上の■のボタンを押して中断しなさい．また，後で Run All でノート全体を実行した時に止まらないように `comb(30, 15)` をコメントアウトしておきなさい．

メモ化を使って `comb(n, k)` を効率化した `fast_comb(n, k)` を実装したい．

しかし今度は，`fib(n)` のメモ化と違って，2つの引数 `n, k` を両方まとめて辞書のキーとして覚えておく必要がある．

リスト `[n, k]` をキーとして `memo[ [n, k] ]` のようにしたくなるが，実はこれはできない．リストは辞書のキーとしては使えないためである．

リスト `[n, k]` の代わりにタプルを使い `(n, k)` とすると辞書のキーとして使える．

（タプルとリストは，ほぼ同じように使えるが，タプルは後から要素を書き換えることができないという制限があり，その代わり辞書のキーとして使える．）

タプルを辞書のキーとして用いる場合も，整数や文字列をキーとする場合と同様に
```python
if (n, k) in memo:
    ...
```
とすればキー `(n, k)` が辞書 `memo` に存在するかどうか判定できて，
```python
memo[(n, k)]
```
とすればキー `(n, k)` に対応する値を取り出せる．

それではメモ化を使って `fast_comb(n, k)` （から呼ばれる `comb_memo`）を実装しなさい：

In [ ]:
def fast_comb(n, k):
    return comb_memo(n, k, {})

def comb_memo(n, k, memo):
    # *** 実装しなさい ***


実装できたら，先ほど中断した ${}_{30}C_{15}$ の計算をしてみなさい：

In [ ]:
%%time
fast_comb(30, 15)

## 練習11.6: 余因子展開による行列式の計算
$n$次の正方行列 $A$ を
$$
A = \left[\begin{array}{ccc}
a_{11} & \cdots & a_{1n} \\
\vdots &        & \vdots \\
a_{n1} & \cdots & a_{nn}
\end{array}\right]
$$

とするとき，$A$ の第 $i$ 行と第 $j$ 列を取り除いた $(n-1)\times(n-1)$ 行列の行列式に $(-1)^{i+j}$ をかけたもの

$$
(-1)^{i+j} \left|\begin{array}{cccccc}
a_{11} & \cdots & a_{1,j-1} & a_{1,j+1} & \cdots & a_{1,n} \\
\vdots &        & \vdots    & \vdots    &        & \vdots \\
a_{i-1,1} & \cdots & a_{i-1,j-1} & a_{i-1,j+1} & \cdots & a_{i-1,n} \\
a_{i+1,1} & \cdots & a_{i+1,j-1} & a_{i+1,j+1} & \cdots & a_{i+1,n} \\
\vdots &        & \vdots    & \vdots    &        & \vdots \\
a_{n,1} & \cdots & a_{n,j-1} & a_{n,j+1} & \cdots & a_{n,n}
\end{array}\right|
$$

を $A$ の $(i, j)$ 余因子というのだった．これを ${\tilde a}_{ij}$ と書くと，$A$ の行列式は

$$
|A| = a_{11} {\tilde a}_{11} + a_{12} {\tilde a}_{12} + \cdots + a_{1n} {\tilde a}_{1n}
$$

と展開できた（余因子展開）．

2重リストで表された行列
```python
m = [[a11, a12, ..., a1n],
     [a21, a22, ..., a2n],
     ...
     [an1, an2, ..., ann]]
```
を受け取り，余因子展開を使ってその行列式を計算する関数 `det(m)` を実装したい．

注：以下では，行番号・列番号は数学のように第 $1$ 行～第 $n$ 行，第 $1$ 列～第 $n$ 列とせずに，
第 $0$ 行～第 $n-1$ 行，第 $0$ 列～第 $n-1$ 列と考える（リストの添え字が 0 からはじまるので）．

`det(m)` の実装方法は色々あるだろうが，再帰を使って余因子展開をそのまま実装するには次のようにするのがよいだろう．

まず行列 `m` の第 $j_1$列, 第$j_2$列, $\dots$, 第$j_k$列（$j_1 < j_2 < \cdots < j_k$ を満たす任意の $k$ 個の列）と，最後の $k$ 行すなわち第 $(n-k)$ 行, 第 $(n-k+1)$ 行, ..., 第 $(n-1)$ 行を取り出し，それらの交差するところの成分からなる $k$ 行 $k$ 列の小行列の行列式を計算する関数 `det_sub(m, [j1, j2, ..., jk])` を，余因子展開を用いて再帰的に実装する．

例えば
```python
m = [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]]
```
ならば，`det_sub(m, [0, 2])` は `m` の第$0$列と第$2$列，および，第$1$行と第$2$行の交差するところの成分を並べた以下の $2\times 2$ 行列の行列式を計算することになる：
```python
[[4, 6],
 [7, 9]]
```

余因子展開を用いると `det_sub(m, [j1, j2, ..., jk])` の値は `m` の行数を `n` として
<!--
```python
  det_sub(m, [j1, j2, j3, ..., jk])
= ((-1) ** 0) * m[n-k][j1] * det_sub(m, [j2, j3, ..., jk]) # [j2, j3, ..., jk] は j1 を取り除いたリスト
+ ((-1) ** 1) * m[n-k][j2] * det_sub(m, [j1, j3, ..., jk]) # [j1, j3, ..., jk] は j2 を取り除いたリスト
...
+ ((-1) ** (k-1)) * m[n-k][jk] * det_sub(m, [j1, j2, j3, ...]) # [j1, j2, j3, ...] は 
                                                               # jk を取り除いたリスト
```
と再帰的に計算できる．（なぜこれでよいか，きちんと納得してから実装を始めること）

-->
<!--
間違ってたバージョン
```python
  det_sub(m, [j1, j2, j3, ..., jk])
= ((-1) ** j1) * m[n-k][j1] * det_sub(m, [j2, j3, ..., jk]) # [j2, j3, ..., jk] は j1 を取り除いたリスト
+ ((-1) ** j2) * m[n-k][j2] * det_sub(m, [j1, j3, ..., jk]) # [j1, j3, ..., jk] は j2 を取り除いたリスト
...
+ ((-1) ** jk) * m[n-k][jk] * det_sub(m, [j1, j2, ..., jk-1]) # [j1, j2, ..., jk-1] は jk を取り除いたリスト
```
と再帰的に計算できる．（なぜこれでよいか，きちんと納得してから実装を始めること）
-->
```python
  det_sub(m, [j1, j2, j3, ..., jk])
= ((-1) **   0  ) * m[n-k][j1] * det_sub(m, [j2, j3, ..., jk]) # [j2, j3, ..., jk] は j1 を取り除いたリスト
+ ((-1) **   1  ) * m[n-k][j2] * det_sub(m, [j1, j3, ..., jk]) # [j1, j3, ..., jk] は j2 を取り除いたリスト
...
+ ((-1) ** (k-1)) * m[n-k][jk] * det_sub(m, [j1, j2, ..., j_{k-1}]) # [j1, j2, ..., j_{k-1}] は
                                                                    # jk を取り除いたリスト
```
と再帰的に計算できる．（なぜこれでよいか，きちんと納得してから実装を始めること）

また，再帰を止める特殊ケースはリスト `[j1, ..., jk]` が長さ1のときで，その場合は
```python
det_sub(m, [j]) --> m[n-1][j]
```
のように残っている 1 x 1 の小行列のただひとつの成分を返せばよい．

`det_sub(m, [j1, j2, ...])` が実装できれば，`det(m)` は `det_sub(m, [0, 1, ..., n-1])` として計算できる．

In [ ]:
def det(m):
    all_cols = [j for j in range(0, len(m))] # = [0, 1, ..., n]
    return det_sub(m, all_cols)

# cols = [j1, j2, ..., jk] とするとき
# m の 第 j1 列, ..., 第 jk 列と
#     第 n-k 行, ..., 第 n-1 行から
# なる小行列の行列式を計算する
def det_sub(m, cols):
    # *** 実装しなさい ***


実装できたらテストしましょう．

まずは 2 x 2 行列：

In [ ]:
i = [[1, 0],
     [0, 1]]
print(det(i) == 1)

m = [[1, 2],
     [3, 4]]
print(det(m) == 1 * 4 - 2 * 3)

つぎは 3 x 3 行列：

In [ ]:
i = [[1, 0, 0],
     [0, 1, 0],
     [0, 0, 1]]
print(det(i) == 1)

j = [[0, 0, 1],
     [0, 1, 0],
     [1, 0, 0]]
print(det(j) == -1)

m = [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]]
print(det(m) == 1*5*9 + 2*6*7 + 3*4*8 - 3*5*7 - 2*4*9 - 1*8*6)

Vandermonde 行列式：

In [ ]:
V = [[1 ** 0, 2 ** 0, 3 ** 0, 4 ** 0, 5 ** 0],
     [1 ** 1, 2 ** 1, 3 ** 1, 4 ** 1, 5 ** 1],
     [1 ** 2, 2 ** 2, 3 ** 2, 4 ** 2, 5 ** 2],
     [1 ** 3, 2 ** 3, 3 ** 3, 4 ** 3, 5 ** 3],
     [1 ** 4, 2 ** 4, 3 ** 4, 4 ** 4, 5 ** 4]]

print(det(V) == 1 * 1 * 1 * 1 * 2 * 2 * 2 * 3 * 3 * 4)

---
**提出前の確認**

おつかれさまでした．以上で今回の必須課題は終わりです．
* メニューの Run から Run All Above Selected Cells を選択してここから上のセルが全て正しく実行されることを確かめ，
* 左上の保存のボタンを押して出力結果を保存してからノートブックを提出してください．

## エクストラ課題11.7: 順列の符号を用いた行列式の計算
前の課題の検算を兼ねて，別の方法でも行列式を計算してみよう．

$n$ 次正方行列 $A = [a_{ij}]$ に対して，$(1, 2, \dots, n)$ の全ての順列の集合を $S = \{(1, 2, \dots, n), (2, 1, \dots, n), \dots\}$ とするとき，$A$ の行列式はもともと

$$
|A| = \sum_{(p_1, p_2, \dots, p_n) \in S} \text{sgn}(p_1, p_2, \dots, p_n) a_{1p_1} a_{2p_2}\cdots a_{np_n}
$$

と定義されるのだった．ここで $\text{sgn}(p_1, p_2, \dots, p_n)$ は順列 $(p_1, p_2, \dots, p_n)$ の**符号**である．

順列 $(p_1, p_2, \dots, p_n)$ の符号は，順列の要素のペア $p_i, p_j \;(i < j)$ のうち小さい順になっていないもの，つまり $i < j$ だが $p_i > p_j$ となっているペアの数を $t$ （転倒数）とするとき $\text{sgn}(p_1, p_2, \dots, p_n) = (-1)^t$ と定義されるのだった．

### Step 1: 順列の符号の計算
$(1, 2, \dots, n)$ を並べ替えた順列 `[p1, p2, ..., pn]` を入力とし，その符号を計算する関数 `sgn([p1, p2, ..., pn])` を実装せよ．

2重ループによって転倒数 $t$ を調べて $(-1)^t$ を返してもよいが，「順列の2つの要素を入れ替えると符号は(-1)倍される」ことと「$(1, 2, \dots, n)$ の符号は 1 である」ことを使うと少し効率よく計算できる．しかし本題ではないのでどっちでもよい．

In [ ]:
# 順列 ps = [p1, p2, ...pn] の
# 符号を計算する
def sgn(ps):
    # *** 実装せよ ***


実装できたらテストしなさい：

In [ ]:
print(sgn([4, 1, 2, 3]) == -1)
print(sgn([1, 4, 2, 3]) == 1)
print(sgn([3, 2, 1, 4]) == -1)

### Step 2: 定義どおりの行列式の計算
少し上で定義した `perm` を使って，`[1, 2, ..., n]` の全ての順列を生成し，それを使って2重リストで表された行列
```python
m = [[a11, ..., a1n],
     [a21, ..., a2n],
     ...
     [an1, ..., ann]]
```
の行列式を計算する関数 `det_as_def(m)` を実装しなさい（`perm(n)` が 1, ..., $n$ の順列を返すのに対し，行番号・列番号は **0から始まる**ことに注意）：

In [ ]:
def det_as_def(m):
    # *** 実装しなさい ***


実装できたらテストしなさい：

In [ ]:
i = [[1, 0],
     [0, 1]]
print(det_as_def(i) == 1)

m = [[1, 2],
     [3, 4]]
print(det_as_def(m) == 1 * 4 - 2 * 3)

i3 = [[1, 0, 0],
      [0, 1, 0],
      [0, 0, 1]]
print(det_as_def(i3) == 1)

j = [[0, 0, 1],
     [0, 1, 0],
     [1, 0, 0]]
print(det_as_def(j) == -1)

m3 = [[1, 2, 3],
      [4, 5, 6],
      [7, 8, 9]]
print(det_as_def(m3) == 1*5*9 + 2*6*7 + 3*4*8 - 3*5*7 - 2*4*9 - 1*8*6)

V = [[1 ** 0, 2 ** 0, 3 ** 0, 4 ** 0, 5 ** 0],
     [1 ** 1, 2 ** 1, 3 ** 1, 4 ** 1, 5 ** 1],
     [1 ** 2, 2 ** 2, 3 ** 2, 4 ** 2, 5 ** 2],
     [1 ** 3, 2 ** 3, 3 ** 3, 4 ** 3, 5 ** 3],
     [1 ** 4, 2 ** 4, 3 ** 4, 4 ** 4, 5 ** 4]]

print(det_as_def(V) == 1 * 1 * 1 * 1 * 2 * 2 * 2 * 3 * 3 * 4)

少し大きな行列で，`det` と `det_as_def` の結果を時間を測りつつ比べてみよう．どっちが速いだろうか？

（正しく実装してあれば，どちらも10秒はかからないであろう）

In [ ]:
large = [[25, 0, 15, 24, 15, 38, 10, 48, 86, 29],
         [12, 85, 98, 70, 69, 15, 79, 86, 7, 24],
         [10, 83, 15, 65, 74, 24, 63, 89, 100, 17],
         [76, 6, 90, 24, 93, 79, 79, 80, 70, 52],
         [21, 94, 73, 73, 24, 88, 48, 39, 43, 47],
         [54, 58, 93, 43, 19, 20, 16, 74, 62, 87],
         [88, 52, 98, 82, 94, 4, 40, 52, 60, 26],
         [2, 69, 32, 7, 81, 45, 25, 54, 31, 87],
         [91, 31, 32, 98, 29, 53, 73, 24, 62, 84],
         [27, 31, 38, 42, 9, 31, 34, 91, 62, 86]]

In [ ]:
%%time
det(large)

In [ ]:
%%time
det_as_def(large)

## エクストラ課題11.8: 行列式計算の効率化
`det(m)` も `det_as_def(m)` も，それほど効率的ではない．

`det_as_def(m)` で計算している $n!$ 個の項の間には部分的に似ているところが沢山ある．これをまとめて計算していけば余因子展開に似た方法になるだろうが，`det(m)` より早くなることはないだろう．

`det` をもっと効率化するにはどうしたらよいだろうか．メモ化は使えるだろうか？

全く違う方法としては，例えば掃き出し法によって三角行列を作り，その対角成分の積として行列式を計算することも考えられる．どっちが速いだろうか？

`det(m)` より速く行列 `m` の行列式の計算を行う `fast_det(m)` を実装しなさい．

上で定義した $10 \times 10$ 行列 `large` に対して `fast_det(large)` が `det(large)` よりも100倍以上速くなるようにすること．

In [ ]:
def fast_det(m):
    # *** 実装しなさい ***


実装できたら `det` の結果と比べてテストしなさい：

In [ ]:
# 1/10000 の誤差まで許容
eps = 0.0001

i = [[1, 0],
     [0, 1]]
print(abs(det(i) - fast_det(i)) < eps)

m = [[1, 2],
     [3, 4]]
print(abs(det(m) - fast_det(m)) < eps)

i3 = [[1, 0, 0],
      [0, 1, 0],
      [0, 0, 1]]
print(abs(det(i3) - fast_det(i3)) < eps)

j = [[0, 0, 1],
     [0, 1, 0],
     [1, 0, 0]]
print(abs(det(j) - fast_det(j)) < eps)

m3 = [[1, 2, 3],
      [4, 5, 6],
      [7, 8, 9]]
print(abs(det(m3) - fast_det(m3)) < eps)

V = [[1 ** 0, 2 ** 0, 3 ** 0, 4 ** 0, 5 ** 0],
     [1 ** 1, 2 ** 1, 3 ** 1, 4 ** 1, 5 ** 1],
     [1 ** 2, 2 ** 2, 3 ** 2, 4 ** 2, 5 ** 2],
     [1 ** 3, 2 ** 3, 3 ** 3, 4 ** 3, 5 ** 3],
     [1 ** 4, 2 ** 4, 3 ** 4, 4 ** 4, 5 ** 4]]

print(abs(det(V) - fast_det(V)) < eps)

上で定義した 10 x 10 行列 `large` の行列式を時間を測りながら計算してみなさい：

In [ ]:
%%time

fast_det(large)

---
**提出前の確認**

おつかれさまでした．以上で今回の課題は全て終わりです．
* メニューの Run から Run All Cells （または解いたエクストラ課題の最後をクリックして Run All Above Selected Cells）を選択してここから上のセルが全て正しく実行されることを確かめ，
* 左上の保存のボタンを押して出力結果を保存してからノートブックを提出してください．

中級編２：おわり